In [157]:
import shelve

# First some setup
We create a hash table of some arbitrary data which
we will then store into our shelf/database.

From then on, we will only access the data via the shelf and not our hash table.

In [158]:
# Build an arbitrary dict
hash_table = {}
for key in range(40):
    # Populate with arbitrary data - note keys are string
    hash_table[str(key)] = {
        "column1": "some data",
        "column2": "some data",
        "column3": {"nested col1": "some other data"}
    }

In [159]:
# Define our shelf file
db_file = "./db/shelf_file"

# Storing and retrieving data in the shelf.

In [160]:
# Store dict data into shelf, context manager (with) closes automatically
# We store the files into a folder /db
with shelve.open(db_file) as shelf:
    for key in hash_table:
        shelf[key] = hash_table[key]

In [161]:
# Loading and reading from the shelf is the same as before.
# "r" specifies read only; by default it's set to "c", which is read/write/create
with shelve.open(db_file, "r") as shelf:
    for key in shelf:
        print(shelf[key])

{'column1': 'some data', 'column2': 'some data', 'column3': {'nested col1': 'some other data'}}
{'column1': 'some data', 'column2': 'some data', 'column3': {'nested col1': 'some other data'}}
{'column1': 'some data', 'column2': 'some data', 'column3': {'nested col1': 'some other data'}}
{'column1': 'some data', 'column2': 'some data', 'column3': {'nested col1': 'some other data'}}
{'column1': 'some data', 'column2': 'some data', 'column3': {'nested col1': 'some other data'}}
{'column1': 'some data', 'column2': 'some data', 'column3': {'nested col1': 'some other data'}}
{'column1': 'some data', 'column2': 'some data', 'column3': {'nested col1': 'some other data'}}
{'column1': 'some data', 'column2': 'some data', 'column3': {'nested col1': 'some other data'}}
{'column1': 'some data', 'column2': 'some data', 'column3': {'nested col1': 'some other data'}}
{'column1': 'some data', 'column2': 'some data', 'column3': {'nested col1': 'some other data'}}
{'column1': 'some data', 'column2': 'som

In [162]:
# Shelve uses pickle to serialise data, so storing python objects works fine:
# Create an arbitrary class
class SomeObject:
    def __init__(self) -> None:
        self.some_property = [
            "a", "b", "c", "d"
        ]
    def __str__(self) -> str:
        return ", ".join(self.some_property)

    def mutate(self, data) -> None:
        self.some_property.append(data)

In [163]:
# Let's add this new class to key 40 of our shelf
with shelve.open(db_file) as shelf:
    shelf["40"] = SomeObject()

    # Check our object
    print(shelf["40"])

a, b, c, d


In [164]:
# Of course, accessing it later works fine
# Shelve lets us load only entry 40 into memory and not the entire file
with shelve.open(db_file) as shelf:
    print(shelf["40"])

a, b, c, d


In [165]:
# If we dont use a context manager (with) then we must close the file manually,
# otherwise we cause a memory leak as it never closes itself.
shelf = shelve.open(db_file)
print(shelf["5"])
shelf.close()

# Using the with context manager is simpler and easier.

{'column1': 'some data', 'column2': 'some data', 'column3': {'nested col1': 'some other data'}}


# Mutating objects in the shelf and using `writeback=True`

In [166]:
# By default, shelve won't mutate entries, only overwrite them
with shelve.open(db_file) as shelf:
    shelf["100"] = [1, 2, 3, 4]
    shelf["100"].append(5)
    print(shelf["100"])

# Note the output is still [1, 2, 3, 4] and not [1, 2, 3, 4, 5]

[1, 2, 3, 4]


In [167]:
# If you need to mutate data instead of simply overwriting, you must enable writeback
with shelve.open(db_file, writeback=True) as shelf:
    print(shelf["100"])
    shelf["100"].append(5)
    print(shelf["100"])

# Use this only if you MUST mutate as this significantly increases the memory overhead

[1, 2, 3, 4]
[1, 2, 3, 4, 5]


In [168]:
# Using the previous example, we can sidestep this problem without writeback=True
with shelve.open(db_file) as shelf:
    print(shelf["100"])
    
    array = shelf["100"]
    array.append(6)
    shelf["100"] = array
    
    print(shelf["100"])

[1, 2, 3, 4, 5]
[1, 2, 3, 4, 5, 6]


In [169]:
# This problem extends to class methods as well
with shelve.open(db_file) as shelf:
    shelf["our class"] = SomeObject()
    print(shelf["our class"])

    shelf["our class"].mutate("X")
    print(shelf["our class"])

# Like before, we'll have to overwrite the class with a copy rather than mutating it.

a, b, c, d
a, b, c, d


# Exceptions and Errors

In [170]:
# Accessing a key that doesn't exist raises a KeyError
with shelve.open(db_file) as shelf:
    try:
        print(shelf["apples"])
    except KeyError as e:
        print("Key: {} does not exist in shelf".format(e))

Key: b'apples' does not exist in shelf


# Deleting data from the shelf

In [171]:
# We can delete an entry with the del keyword
with shelve.open(db_file) as shelf:
    shelf["some entry"] = "This entry will be deleted"
    print(shelf["some entry"])

    del shelf["some entry"]

    try:
        print(shelf["some entry"])
    except KeyError as e:
        print("{} doesn't exist anymore".format(e))

This entry will be deleted
b'some entry' doesn't exist anymore


In [172]:
# Cleaning up our objects
with shelve.open(db_file) as shelf:
    del shelf["our class"]
    del shelf["40"]